In [1]:
#Importing the libraries
from pprint import pprint
import requests
import http.client
import json
import pandas as pd
from datetime import datetime as dt

In [2]:
# Creating the API call
conn = http.client.HTTPSConnection("www.jambase.com")

headers = { 'Accept': "application/json" }

conn.request("GET", "/jb-api/v1/events?geoCityId=jambase%3A4219129&apikey=964b3dbc-0ea0-46b2-bbf9-476d3cf9ab18", headers=headers)

res = conn.getresponse()
data = res.read()

decoded_data = data.decode("utf-8")

# Use the json module to load the JSON data
json_data = json.loads(decoded_data)

# Print the JSON data using pprint (optional)
from pprint import pprint
pprint(json_data)

# Export the JSON data to a file (e.g., 'output.json')
with open('output.json', 'w') as json_file:
    json.dump(json_data, json_file)

# Close the connection
conn.close()

{'events': [{'@type': 'Concert',
             'dateModified': '2023-10-20T13:11:24Z',
             'datePublished': '2023-07-27T20:01:35Z',
             'doorTime': '',
             'endDate': '2023-11-20',
             'eventAttendanceMode': 'live',
             'eventStatus': 'scheduled',
             'identifier': 'jambase:10816636',
             'image': 'https://www.jambase.com/wp-content/uploads/2023/01/fit-for-an-autopsy-1480x832.png',
             'isAccessibleForFree': False,
             'location': {'@type': 'Venue',
                          'address': {'@type': 'PostalAddress',
                                      'addressCountry': {'@type': 'Country',
                                                         'alternateName': 'USA',
                                                         'identifier': 'US',
                                                         'name': 'United '
                                                                 'States'},
                

In [3]:
# Checking the API pull for the event names
n=0
for i in range(0,len(json_data['events'])):
    n+=1
    print(f"{n}: {json_data['events'][i]['name']}")

1: Fit For An Autopsy at Vibes Event Center
2: Mike Dimes at Paper Tiger
3: Corb Lund at Sam’s Burger Joint
4: DeeBaby at Paper Tiger
5: Intocable at Majestic Theatre
6: Anything Box at Sam’s Burger Joint
7: Purple Kiss at Vibes Event Center
8: Jonny Craig at The Rock Box
9: Pussy Riot at Paper Tiger
10: Slaid Cleaves at Sam’s Burger Joint
11: Kidz Bop at Majestic Theatre
12: Wild Party at Paper Tiger
13: Patricia Vonne at Sam’s Burger Joint
14: Cancamusa at Paper Tiger
15: Donovan Keith at Sam’s Burger Joint
16: Brit Floyd at The Aztec Theatre
17: Angel Du$t at Paper Tiger
18: Andrés at Vibes Event Center
19: INOHA at Sam’s Burger Joint
20: Andrea Bocelli at Frost Bank Center
21: Jeff Rosenstock at Paper Tiger
22: Jaret Ray Reddick at Sam’s Burger Joint
23: We Are Scientists at Paper Tiger
24: The Meteors at Vibes Event Center
25: Jason Eady at Sam’s Burger Joint
26: Frozen Soul at The Rock Box
27: Nascar Aloe at Studio 210 Nightclub and Bar
28: Lauren Daigle at Frost Bank Center
29: 

In [86]:
#Parsing the event info

# Creating empty lists to append parsed information to.
headliner_name=[]
opener_name=[]
artist_genre=[]
event_date=[]
event_venue=[]
event_status=[]
venue_address=[]
ticket_link=[]
# Define the condition (e.g., if the value is positive)
condition = lambda x: x is True


# Looping through events and extracting relevant info.
for event in range(0,len(json_data['events'])):

    #Skipping non-concert events
    if json_data['events'][event]['@type']!='Concert':
        pass
    else:

        #grabbing the headliner's name
        for artist in range(len(json_data['events'][event]['performer'])):
            if json_data['events'][event]['performer'][artist]['x-isHeadliner'] is True:
                headliner_name.append(json_data['events'][event]['performer'][artist]['name'])
            else:
                pass
            
        event_date.append(json_data['events'][event]['endDate'])
        event_venue.append(json_data['events'][event]['location']['name'])
        event_status.append(json_data['events'][event]['eventStatus'])
        #Creating a composite address field
        venue_address.append(json_data['events'][event]['location']['address']['streetAddress']+', '+ 
                             json_data['events'][event]['location']['address']['addressLocality']+' '+
                             json_data['events'][event]['location']['address']['addressRegion']['name']+' '+
                             json_data['events'][event]['location']['address']['postalCode'])
        # We are assigning genre based on the headliner
        artist_genre.append(json_data['events'][event]['performer'][0]['genre'])
        # We are only taking the first ticketing url here
        ticket_link.append(json_data['events'][event]['offers'][0]['url'])

In [88]:
print(len(headliner_name), len(opener_name), len(artist_genre), len(event_date), len(event_status), len(event_venue), len(venue_address))


40 0 40 40 40 40 40


In [89]:
print(headliner_name, opener_name)

['Fit For An Autopsy', 'Mike Dimes', 'Corb Lund', 'DeeBaby', 'Intocable', 'Anything Box', 'Purple Kiss', 'Jonny Craig', 'Pussy Riot', 'Slaid Cleaves', 'Kidz Bop', 'Wild Party', 'Patricia Vonne', 'Cancamusa', 'Donovan Keith', 'Brit Floyd', 'Angel Du$t', 'Andrés', 'INOHA', 'Andrea Bocelli', 'Jeff Rosenstock', 'Jaret Ray Reddick', 'We Are Scientists', 'The Meteors', 'Jason Eady', 'Frozen Soul', 'Nascar Aloe', 'Lauren Daigle', 'Brett Young', 'Suffocation', 'Prayers', 'Texas Flood – a Tribute to Stevie Ray Vaughan', 'Mariachi Nuevo Tecalitlan', 'Tommy Emmanuel', 'Mark Searcy', 'Blood Club', 'Devora', 'Marie Osmond', 'Silverstein', 'Early Moods'] []


In [90]:
# Adding an 'up-to-date as of' field
last_run=dt.today().date()

#Creating the structured pandas dataset
all_concerts=pd.DataFrame({'headliner_name':headliner_name,
                           
                           'genre': artist_genre, 
                           'event_date':event_date,
                           'event_status':event_status,
                           'event_venue_nm':event_venue, 
                           'venue_address':venue_address, 
                           'up_to_date_as_of':last_run})


all_concerts.head()

,headliner_name,genre,event_date,event_status,event_venue_nm,venue_address,up_to_date_as_of
0,Fit For An Autopsy,"[metal, punk]",2023-11-20,scheduled,Vibes Event Center,"1211 East Houston, San Antonio Texas 78205",2023-11-20
1,Mike Dimes,[hip-hop-rap],2023-11-20,scheduled,Paper Tiger,"2410 N St Mary's St, San Antonio Texas 78212",2023-11-20
2,Corb Lund,"[americana, country-music, folk, rock]",2023-11-21,scheduled,Sam’s Burger Joint,"330 East Grayson Street, San Antonio Texas 78215",2023-11-20
3,DeeBaby,[],2023-11-21,scheduled,Paper Tiger,"2410 N St Mary's St, San Antonio Texas 78212",2023-11-20
4,Intocable,"[latin, pop]",2023-11-22,scheduled,Majestic Theatre,"226 East Houston St., San Antonio Texas 78205",2023-11-20


In [21]:
# Checking logic for artist name

for artist in range(0,len(json_data['events'][event]['performer'])):
    print(json_data['events'][event]['performer'][artist]['x-isHeadliner'])


True
False
False


# Spotify API request:

#IMPORTANT: API resets after one hour of use!

import requests

url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type": "client_credentials",
    "client_id": "22ba93d78b104ade8f90c67ec7cd1e35",
    "client_secret": "7c3fb228e7d94fd7808614993728843e"
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print("Access Token:", access_token)
    #access_token='Bearer '+ access_token
else:
    print("Failed to retrieve access token. Status code:", response.status_code)



import random
import string

# Define the client_id, redirect_uri, and scope
client_id = '22ba93d78b104ade8f90c67ec7cd1e35'
redirect_uri = 'http://localhost:8080'
scope = 'user-follow-read'

# Generate a random state string
stateKey = 'state_key'  # Assuming you have a variable for stateKey
state = ''.join(random.choice(string.ascii_letters) for _ in range(16))
# You can use the 'stateKey' variable for localStorage as well, similar to the original code

# Build the URL
url = 'https://accounts.spotify.com/authorize'
url += '?response_type=token'
url += f'&client_id={client_id}'
url += f'&scope={scope}'
url += f'&redirect_uri={redirect_uri}'
url += f'&state={state}'

print(url)

# Define the URL and headers
url='https://api.spotify.com/v1/me/following?type=artist'
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make the GET request using the requests library
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Request was successful
    data = response.json()  # Parse the response as JSON
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)  # Print the response content for debugging
